In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# 1. 데이터셋 로드 및 전처리: 데이터로더 만들기
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# 각 로더는 inputs, targets 순으로 반환하며 inputs의 shape는 batch_size x 3 x 32 x 32, targets의 shape는 batch_size임
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# train_loader: 미니배치 크기 64, shuffle 이용, num_workers=2 작성
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers = 2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# test_loader: 미니배치 크기 64, shuffle 이용x, num_workers=2 작성
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False, num_workers = 2)

100%|██████████| 170M/170M [00:03<00:00, 43.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# 2. 모델 정의: 모델 정의 수행
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #작성
        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        #작성
        logits = self.conv_relu_stack(x)
        return logits

In [ ]:
# 3. 학습 설정: 학습 설정 수행 (cuda 활용, 손실 함수로는 교차 엔트로피 오차 (cross entropy loss), optimizer는 Adam으로 학습률은 0.001의 값 활용)
#작성
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

cuda


In [ ]:
# 4. 학습 루프: 학습 루프 만들기
def train(model, loader, criterion, optimizer, device):
    #작성
    size = len(loader.dataset)
    loss_sum = 0
    current = 0
    for batch, (x, y) in enumerate(loader):
      x, y = x.to(device), y.to(device)

      pred = model(x)
      #print(pred)
      loss = criterion(pred, y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_sum += loss.item()
      current += (pred.argmax(1) == y).type(torch.float).sum().item()

    #loss = loss.item()

    loss_sum /= batch
    current /= size
    return loss_sum, current

In [ ]:
# 5. 테스트 루프: 테스트 루프 만들기
def evaluate(model, loader, criterion, device):
  #작성
  size = len(loader.dataset)

  num_batches = len(loader)
  model.eval()
  test_loss, correct = 0, 0

  with torch.no_grad():
    for x, y in loader:
      x, y = x.to(device), y.to(device)
      pred = model(x)
      test_loss += criterion(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  return test_loss, correct

In [ ]:
# 6. 학습 및 테스트: 실행
num_epochs = 200
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

print("Training Complete")

Epoch 1/200
Train Loss: 0.9353, Train Accuracy: 0.6697
Test Loss: 0.9448, Test Accuracy: 0.6668
Epoch 2/200
Train Loss: 0.8393, Train Accuracy: 0.7036
Test Loss: 0.8316, Test Accuracy: 0.7068
Epoch 3/200
Train Loss: 0.7731, Train Accuracy: 0.7287
Test Loss: 0.8271, Test Accuracy: 0.7089
Epoch 4/200
Train Loss: 0.7292, Train Accuracy: 0.7427
Test Loss: 0.7484, Test Accuracy: 0.7378
Epoch 5/200
Train Loss: 0.6829, Train Accuracy: 0.7594
Test Loss: 0.7173, Test Accuracy: 0.7526
Epoch 6/200
Train Loss: 0.6508, Train Accuracy: 0.7710
Test Loss: 0.6952, Test Accuracy: 0.7575
Epoch 7/200
Train Loss: 0.6219, Train Accuracy: 0.7811
Test Loss: 0.7088, Test Accuracy: 0.7572
Epoch 8/200
Train Loss: 0.6023, Train Accuracy: 0.7892
Test Loss: 0.6885, Test Accuracy: 0.7638
Epoch 9/200
Train Loss: 0.5770, Train Accuracy: 0.7991
Test Loss: 0.6581, Test Accuracy: 0.7737
Epoch 10/200
Train Loss: 0.5552, Train Accuracy: 0.8053
Test Loss: 0.6677, Test Accuracy: 0.7749
Epoch 11/200
Train Loss: 0.5415, Train 

KeyboardInterrupt: 



---



In [ ]:
#교수님 해설

self.seq1 = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
    nn.ReLU()
    nn.MaxPool2d(kernel_size = 2, stride = 2)
    nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
    nn.ReLU()
    nn.MaxPool2d(kernel_size = 2, stride = 2)
    nn.Conv2d(64, 128, kernel_size = 3, padding = 1)
    nn.ReLU()
    nn.MaxPool2d(kernel_size = 2, padding = 2)
)
self.seq2 = nn.Sequential(
    nn.Flatten()
    nn.Linear(128 * 4 * 4, 256)
    nn.ReLU()
    nn.Dropout(0.5)
    nn.Linear(256, 10)
)

In [ ]:
# 5. 테스트 루프: 테스트 루프 만들기
def evaluate(model, loader, criterion, device):
  #작성
  model.train()
  total_loss = 0
  correct = 0
  for inputs, targets in loader:
    inputs, targets = inputs.to(device), targets.to(device) #to.(device)를 통해 cuda로 보낸다.

    #forward
    outputs = model(inputs) #순전파 진행(input을 모델에 통과시킨다)
    loss = criterion(outputs, targets)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, predicted = outputs.max(outputs, 1)
    correct += (predicted == targets).sum().item()

  return total_loss / len(loader), correct / len(loader.dataset)

In [ ]:
# 5. 테스트 루프: 테스트 루프 만들기
def evaluate(model, loader, criterion, device):
  #작성
  model.eval()
  total_loss = 0
  correct = 0
  with torch.no_grad(): #역전파를 수행하지 않아도 되니까
    for inputs, targets in loader:
      inputs, targets = inputs.to(device), targets.to(device)

      #forward
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      total_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == targets).sum().item()

  return total_loss / len(loader), correct / len(loader.dataset)